<a href="https://colab.research.google.com/github/AishwaryaNambissan/Code/blob/main/LDA_FinalDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installations and Packages

In [1]:
import pandas as pd
from gensim.utils import simple_preprocess
from gensim import corpora
from gensim import models

%pip install ijson
import ijson

import pandas as pd

from gensim.parsing.preprocessing import preprocess_string
from gensim.parsing.preprocessing import strip_multiple_whitespaces
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import strip_punctuation
from gensim.parsing.preprocessing import strip_tags
from gensim.parsing.preprocessing import strip_numeric
from gensim.parsing.preprocessing import strip_non_alphanum
from gensim.parsing.preprocessing import strip_short
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.preprocessing import STOPWORDS
from nltk.tokenize import word_tokenize

import gensim
from gensim import corpora
from gensim.models.coherencemodel import CoherenceModel
import matplotlib.pyplot as plt

from collections import Counter
import csv
from wordcloud import WordCloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 2.1 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


Printing out information about the dataset

In [4]:
file =  '/content/gdrive/MyDrive/Colab Notebooks/extracted_files/json/artificial_intelligence.json'
affs = set()
author_names = set()
content = []
titles=[]
counter = 0
try:
    with open(file, 'r') as f:
        # Parse the JSON objects one by one
        parser = ijson.items(f, 'item')

        # Iterate over the JSON objects
        for item in parser:
          counter = counter + 1
          try:
            # Process each JSON object as needed
            if 'abstract' in item.keys() and 'title' in item.keys():
              title = ' '.join(item['title'])
              content.append(item['abstract']+ " " + title)
              titles.append(title)
            if 'author' in item.keys():
              authors = item['author']
              for val in authors:
                # Both given and family names have to exist to be counted
                if 'given' in val.keys() and 'family' in val.keys():
                # Extract the name of the author from the given and family names
                  name = val['given'] + " " + val['family']
                # Add the author's name to the set of authors
                  author_names.add(name)

                if 'affiliation' in val.keys():
                  # Extract the name of the author
                  affiliations = val['affiliation']
                  if len(affiliations) > 0:
                    for aff in affiliations:
                      if 'name' in aff.keys():
                        name = aff['name']
                        # Add the author's affiliation to the set of affiliations
                        affs.add(name)

          except Exception as e:
              print("Error processing JSON item:", str(e))
except FileNotFoundError:
  print("File not found:", file)
except Exception as e:
  print("Error reading file:", str(e))

print("Number of records", counter)
print("Number of affiliations:", len(affs))
print("Number of co-authors:", len(author_names))


Number of records 501629
Number of affiliations: 163722
Number of co-authors: 736857


Creating the dataframe with required feature


In [6]:
df = pd.DataFrame(columns=['content', 'title'])
# Convert JSON to DataFrame Using read_json()
df['content'] = pd.DataFrame(content)
df['title'] = pd.DataFrame(titles)


# Checking for missing values
df.isnull().sum()

# Dropping those columns with null values for abstract or title
df = df.dropna()
df.shape
df.head()

,content,title
0,<jats:p> Artificial immune system is inspired ...,IMMUNOLOGY AND ARTIFICIAL IMMUNE SYSTEMS
1,<jats:p> Accurate time series forecasting are ...,TIME SERIES FORECASTING. A COMPARATIVE STUDY B...
2,<jats:p> This paper addresses the problem of e...,Artificial Immune System-Based Learning Style ...
3,<jats:p> Recent trends in educational technolo...,Artificial Intelligence in Psychomotor Learnin...
4,<jats:p> As the dairy products have a short co...,Application of Improved Artificial Intelligenc...


Preprocessing and Cleaning the data

In [7]:
CUSTOM_FILTERS = [lambda x: x.lower(), strip_tags, strip_punctuation, strip_multiple_whitespaces, strip_numeric, strip_non_alphanum,  lambda x: strip_short(x, minsize=4)]

filtered_text = []
for item in df['content'].astype(str):
  filtered_text.append(preprocess_string(item, CUSTOM_FILTERS))


Lemmatization

In [8]:
import nltk

nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [9]:
lemmatized_words = [[lemmatizer.lemmatize(word) for word in sublst] for sublst in filtered_text]


In [10]:
print("One row of Lemmatized words")
print(lemmatized_words[0])

One row of Lemmatized words
['artificial', 'immune', 'system', 'inspired', 'natural', 'immune', 'system', 'solving', 'computational', 'problem', 'immunological', 'principle', 'that', 'primarily', 'used', 'artificial', 'immune', 'system', 'clonal', 'selection', 'principle', 'immune', 'network', 'theory', 'negative', 'selection', 'mechanism', 'these', 'principle', 'have', 'been', 'applied', 'anomaly', 'detection', 'pattern', 'recognition', 'computer', 'network', 'security', 'dynamic', 'environment', 'learning', 'robotics', 'data', 'analysis', 'optimization', 'scheduling', 'timetabling', 'this', 'paper', 'describes', 'these', 'three', 'immunological', 'principle', 'were', 'adapted', 'previous', 'researcher', 'their', 'artificial', 'immune', 'system', 'model', 'algorithm', 'finally', 'application', 'various', 'artificial', 'immune', 'system', 'various', 'domain', 'summarized', 'time', 'line', 'immunology', 'artificial', 'immune', 'system']


Removing stop words

In [11]:
nltk.download('punkt')

stop_words = STOPWORDS.union(set(['jats', 'ai', 'artificial', 'intelligence', 'research','technology','article',
                                  'development', 'paper', 'application', 'method','deep','algorithm', 'problem', 'model', 'iso', 'function',
                                  'noise', 'sample', 'layer', 'optimization', 'solution',
                                  'optimal', 'parameter', 'predict', 'forecast', 'cluster', 'classifier', 'label', 'class',
                                  'test', 'training', 'case', 'level', 'give', 'focus',
                                  'real', 'however', 'review', 'need', 'various', 'many', 'solve', 'analyze', 'human',
                                  'include', 'datasets','dataset','image', 'approach', 'accuracy', 'information',
                                  'also', 'time', 'show',  'provide', 'task', 'make',
                                  'performance', 'knowledge', 'user', 'feature', 'proposed','propose',
                                  'process', 'system', 'well', 'study',
                                  'different', 'high', 'improve', 'design', 'recognition',
                                  'train', 'compare', 'field', 'apply', 'develop', 'based',
                                  'technique', 'analysis', 'present', 'work',
                                  'learn', 'machine', 'language', 'large',
                                  'intelligent', 'base', 'neural', 'network', 'framework','learning',
                                  'result', 'use', 'supervised', 'representation', 'domain', 'architecture',
                                  'prediction', 'classification', 'natural', 'conference', 'data',
                                  'conclusion', 'objective', 'abstract','background', 'author', 'disclosure', 'title', 'computer', 'agent',
                                  'challenge','processing', 'future', 'recent', 'transfer', 'state', 'yang']))


s = ""
print(stop_words)
cleaned_text_list = []
for line in lemmatized_words:
  s = " ".join(line)
  tokenized_text = word_tokenize(s)
  cleaned_text = [word for word in tokenized_text if not word in stop_words]
  cleaned_text_list.append(cleaned_text)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


frozenset({'seem', 'whose', 'present', 'between', 'mill', 'whether', 'from', 'km', 'used', 'meanwhile', 'user', 'several', 'once', 'up', 'at', 'myself', 'why', 'somewhere', 'towards', 'call', 'show', 'because', 'ourselves', 'herself', 'wherein', 'eight', 'amoungst', 'whereafter', 'whereby', 'elsewhere', 'only', 'then', 'recognition', 'optimization', 'etc', 'prediction', 'not', 'hers', 'to', 'ltd', 'approach', 'which', 'one', 'human', 'whereas', 'however', 'thereby', 'feature', 'who', 'afterwards', 'system', 'others', 'abstract', 'off', 'seems', 'empty', 'include', 'cry', 'interest', 'paper', 'themselves', 'thereafter', 'are', 'hasnt', 'time', 'knowledge', 'deep', 'his', 'latterly', 'see', 'whereupon', 'within', 'framework', 'nine', 'technology', 'enough', 'regarding', 'work', 'unless', 'throughout', 'challenge', 'yourself', 'us', 'third', 'each', 'against', 'four', 'five', 'supervised', 'upon', 'i', 'quite', 'propose', 'every', 'improve', 'thin', 'solve', 'whenever', 'processing', 'own

Creating vocabulary dictionary and document term matrix

In [12]:
dictionary = corpora.Dictionary(cleaned_text_list)
doc_term_matrix = [dictionary.doc2bow(con) for con in cleaned_text_list]
print(doc_term_matrix[0])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 7), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 4), (21, 1), (22, 1), (23, 1), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1)]


Finding optimal number of topics using coherence score

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=num_topics, id2word=dictionary)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=dictionary, corpus=doc_term_matrix, texts=cleaned_text_list, start=2, limit=20, step=1)


In [ ]:
# Show graph
limit=20; start=2; step=1;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
max_cv = max(coherence_values)
m_with_max_cv = None


# Print the coherence scores and m with max cv in the range 7 to 19
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))
    if 7 <= m <= 19:
        if max_cv is None or cv >= max_cv:
            max_cv = cv
            m_with_max_cv = m

print("Optimum number of topics: m with max cv:", m_with_max_cv)

In [ ]:
# Optimum number of topics is m_with_max_cv
num_topics = m_with_max_cv
# Creating the object for LDA model using gensim library
LDA = gensim.models.ldamodel.LdaModel

# Build LDA model
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=num_topics, random_state=100,
                chunksize=1000, passes=50,iterations=100)


In [ ]:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()


Perplexity and Coherence score of the model

In [ ]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix,total_docs=10000))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized_words, dictionary=dictionary , coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Visualization

In [ ]:
# Get the dominant topic and its associated cluster for each document
cluster_labels = []
for doc in doc_term_matrix:
    doc_topics = lda_model.get_document_topics(doc)
    dominant_topic = max(doc_topics, key=lambda x: x[1])[0]
    cluster_labels.append(dominant_topic)


# Count the number of documents in each cluster and display
cluster_counts = Counter(cluster_labels)
for key, value in cluster_counts.items():
    print(key, ":", value)


In [ ]:
# Get the cluster labels and counts.
cluster_labels, cluster_counts = list(cluster_counts.keys()), list(cluster_counts.values())

# Create a bubble chart of the cluster counts, with the size of the bubbles proportional to the number of documents in each cluster.
plt.figure(figsize=(15, 15))
plt.scatter(cluster_labels, cluster_counts, s=cluster_counts, c='orange', alpha=0.75)

# Add a title and labels to the plot.
plt.title('Bubble Chart of Cluster Counts')
plt.xlabel('Cluster Label')
plt.ylabel('Number of Documents')

# Show the plot.
plt.show()

In [ ]:
# LDA does soft clustering. This shows all the topics each article is assigned to
topic_assignments = []
for doc in doc_term_matrix:
    topic_dist = lda_model.get_document_topics(doc)
    topic_assignments.append([t[0] for t in topic_dist])

In [ ]:
# To get the topic that an article has maximum probability of being assigned to
dominant_assignments = []

for doc in doc_term_matrix:
      topic_distribution = lda_model.get_document_topics(doc)
      topic_distribution.sort(key=lambda x: x[1], reverse=True)
      dominant_topic = max(topic_distribution, key=lambda item: item[1])[0]
      dominant_assignments.append(dominant_topic)

In [ ]:
article_topic_map = {}
for i, article in enumerate(df['title']):
    dominant_topic = dominant_assignments[i]
    if article_topic_map.get(dominant_topic) is None:
        article_topic_map[dominant_topic] = []
    article_topic_map[dominant_topic].append(article)

In [ ]:
# Specify the output CSV file path
output_file = '/content/gdrive/MyDrive/Colab Notebooks/Topic_Article_Map.csv'

# Open the file in 'w' mode, create a CSV writer object
with open(output_file, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Write the header
    writer.writerow(['Topic', 'Title'])

    # Write the contents of the dictionary
    for key, value in article_topic_map.items():
        writer.writerow([key, value])

print(f"Data written to '{output_file}' successfully.")

Word Cloud

In [ ]:
for t in range(lda_model.num_topics):
    topic_words = lda_model.show_topic(t, 200)
    wordcloud_data = {word: prob for word, prob in topic_words}

    plt.figure()
    plt.imshow(WordCloud().fit_words(wordcloud_data))
    plt.axis("off")
    plt.title("Topic #" + str(t))
    plt.show()
